In [ ]:
import pandas as pd
import finterstellar as fs

In [ ]:
path = './data/'
cd = 'S&P 500'

In [ ]:
file_name = path + cd + ' Historical Data.csv'
df = pd.read_csv(file_name, index_col='Date')

# 날짜 처리
ld = fs.LoadData()
df = ld.date_formatting(df)    
# date_formatting(데이터프레임) - 날짜 처리

df = ld.price_df_trimming(df, cd)
# price_df_trimming(데이터프레임) - 시세 처리
df.head()

In [ ]:
trd = fs.SingleAsset()   # fs 라이브러리의 SingleAsset 모듈을 불러와 trd에 로딩

base_date = '2018-01-01'   # 기준일자 설정
base_date = trd.check_base_date(df, base_date)
# check_base_date(데이터프레임, 기준일자) - 기준일자가 데이터프레임에 들어있는지 체크
base_date

In [ ]:
n = 20     # 평균주가계산 기준일수
sigma = 2     # 편차구간 지정 (시그마의 배수로)

In [ ]:
# 볼린저밴드 계산
bb = trd.bollinger_band(df, cd, n, sigma)
# bollinger_band(데이터프레임, 분석대상코드, 평균기준일, 편차구간) - 볼린저밴드 분석값 계산
bb.tail()

In [ ]:
# 샘플링
sample = bb.loc[base_date:]     # 계산 결과를 기준일자 이후만 잘라내 sample 데이터프레임에 저장
sample.head()

In [ ]:
book = trd.create_trade_book(sample, cd)
# create_trade_book(데이터프레임, [종목코드]) - 트레이딩북 생성
book.head()

In [ ]:
# 트레이딩 전략 설정
thd = 'ub'   # 종목 매도 기준 (편차구간상단:ub, 편차구간중심:center)
buy = 'out'    # 종목 매수 기준 (편차구간 진입 시:in, 편차구간 벗어날 시:out)
book = trd.tradings(sample, book, thd, cd, buy, short=False)
# tradings(데이터프레임, 트레이딩북, 매도기준, 종목코드, 매입기준) - 매매전략을 이용한 백테스팅 수행

In [ ]:
# 포지션 계산
book = trd.position(book, cd)
# position(트레이딩북, 종목코드) - 트레이딩 전략 수행에 수반되는 포지션 판단
'''
    z : zero
    l : long
    s : short
    zz : zero to zero
    zl : zero to long
    lz : long to zero
'''
book['2018-02-01':'2018-02-20']

In [ ]:
# 수익률 계산
fund_rtn = trd.returns(book, cd, display=True)
# returns(트레이딩북, 종목코드) - 전략의 수익률 계산

In [ ]:
# 벤치마크 수익률
bm_rtn = trd.benchmark_return(book, cd)
# benchmark_return(트레이딩북, 종목코드) - 벤치마크 수익률 계산

In [ ]:
# 초과 수익률
exs_rtn = trd.excess_return(fund_rtn, bm_rtn)
# excess_return(전략수익률, 벤치마크수익률) - 초과수익률 계산

그래프로 표현하기

In [ ]:
v = fs.Visualize()    # fs 라이브러리의 Visualize() 모듈을 불러와 v로 지정

In [ ]:
v.BB_trend_view(sample, cd, (15,5))
# v 모듈의 bb_trend_view(데이터프레임, 편차구간, 종목코드, (사이즈)) 함수 - 볼린저밴드 그래프 그리기

In [ ]:
v.position_view(book, cd)
# v 모듈의 position_view(트레이딩북, 종목코드, (사이즈)) 함수 - 보유내역 그래프 그리기

최근 전략

In [ ]:
last_date = sample.index[-1].date().strftime('%Y-%m-%d')
# 현재 투자전략 추출을 위해 데이터 상 최종일 추출
last_date

In [ ]:
trd.trading_strategy(sample, thd, cd, last_date)
# trading_strategy(데이터프레임, 매도기준, 종목코드, 최종일) - 트레이딩 전략 판단

In [ ]:
trd.position_strategy(book, cd, last_date)
# position_strategy(트레이딩북, 종목코드 최종일) - 포지션 구축 전략 판단

백테스팅 결과인 트레이딩북을 한번 구경해볼까요?

In [ ]:
book['2018-02-01':]